This code will help to convert cvs to Json

In [63]:
# Dependencies and Setup
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from datetime import datetime

In [64]:
csv_file = "Raw data/covid19_italy_province.csv"
ItalyData_df = pd.read_csv(csv_file)
ItalyData_df.head()

,SNo,Date,Country,RegionCode,RegionName,ProvinceCode,ProvinceName,ProvinceAbbreviation,Latitude,Longitude,TotalPositiveCases
0,0,2020-02-24T18:00:00,ITA,13,Abruzzo,66,L'Aquila,AQ,42.351222,13.398438,0
1,1,2020-02-24T18:00:00,ITA,13,Abruzzo,67,Teramo,TE,42.658918,13.704400,0
2,2,2020-02-24T18:00:00,ITA,13,Abruzzo,68,Pescara,PE,42.464584,14.213648,0
3,3,2020-02-24T18:00:00,ITA,13,Abruzzo,69,Chieti,CH,42.351032,14.167546,0
4,4,2020-02-24T18:00:00,ITA,13,Abruzzo,979,In fase di definizione/aggiornamento,NaN,NaN,NaN,0


In [65]:
new_ItalyData_df = ItalyData_df[['Date', 'TotalPositiveCases', 'RegionName']].copy()
new_ItalyData_df.tail()

,Date,TotalPositiveCases,RegionName
40196,2020-12-06T17:00:00,24142,Veneto
40197,2020-12-06T17:00:00,31073,Veneto
40198,2020-12-06T17:00:00,4577,Veneto
40199,2020-12-06T17:00:00,2809,Veneto
40200,2020-12-06T17:00:00,403,Veneto


In [66]:
new_ItalyData_df.dropna()

,Date,TotalPositiveCases,RegionName
0,2020-02-24T18:00:00,0,Abruzzo
1,2020-02-24T18:00:00,0,Abruzzo
2,2020-02-24T18:00:00,0,Abruzzo
3,2020-02-24T18:00:00,0,Abruzzo
4,2020-02-24T18:00:00,0,Abruzzo
...,...,...,...
40196,2020-12-06T17:00:00,24142,Veneto
40197,2020-12-06T17:00:00,31073,Veneto
40198,2020-12-06T17:00:00,4577,Veneto
40199,2020-12-06T17:00:00,2809,Veneto


In [67]:
new_ItalyData_df['FormatedDate'] =  pd.to_datetime(new_ItalyData_df['Date'], format='%Y-%m-%d %H:%M:%S')
new_ItalyData_df['Date'] =  new_ItalyData_df['FormatedDate'].dt.strftime('%x')

DB_ItalyData_df = new_ItalyData_df[['Date', 'TotalPositiveCases', 'RegionName']].copy()
DB_ItalyData_df = DB_ItalyData_df.rename(columns={'Date':'recordeddate','TotalPositiveCases':'positivenumber','RegionName':'regionname'})
DB_ItalyData_df.tail()

,recordeddate,positivenumber,regionname
40196,12/06/20,24142,Veneto
40197,12/06/20,31073,Veneto
40198,12/06/20,4577,Veneto
40199,12/06/20,2809,Veneto
40200,12/06/20,403,Veneto


In [68]:
json_file = "Raw data/us_covid19_daily.json"
USdata_df = pd.read_json(json_file, lines=True)
USdata_df.tail()

,date,states,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,...,totalTestResults,lastModified,total,posNeg,deathIncrease,hospitalizedIncrease,negativeIncrease,positiveIncrease,totalTestResultsIncrease,hash
315,20200126,2,0,0,,,,,,,...,2,2020-01-26T24:00:00Z,0,0,0,0,0,0,0,69c67c81e64d10a5270497fc5336779c06fb812b
316,20200125,2,0,0,,,,,,,...,2,2020-01-25T24:00:00Z,0,0,0,0,0,0,0,9ea5731f5b0a79617b994d6842eecfdbeb1dde35
317,20200124,2,0,0,,,,,,,...,2,2020-01-24T24:00:00Z,0,0,0,0,0,0,0,287531342bc8d5f2650b0cc12ebcca3807a64b7b
318,20200123,2,0,0,,,,,,,...,2,2020-01-23T24:00:00Z,0,0,0,0,0,0,1,e0563aac8e1bc7b8b8059c52831b97179ff6f981
319,20200122,2,0,0,,,,,,,...,1,2020-01-22T24:00:00Z,0,0,0,0,0,0,0,e0caa1c66c4aa83baf227afe12261a1493fc0d7e


In [74]:
new_USData_df = USdata_df[['date', 'positive', 'negative','pending']].copy()

new_USData_df.replace('', 0, inplace=True) 


#new_USData_df['FormatedDate'] =  pd.to_datetime(new_USData_df['date'], format='%Y-%m-%d %H:%M:%S')
#new_USData_df['date'] =  new_USData_df['FormatedDate'].dt.strftime('%x')
#new_USData_df['FormatedDate'] =  datetime.strptime(new_USData_df['date'], '%Y%m%d')
new_USData_df['date'] =  new_USData_df['date'].apply(lambda x: pd.to_datetime(str(x), format='%Y%m%d'))


new_USData_df = new_USData_df.rename(columns={'date':'recordeddate','positive':'positivenumber','negative':'negativenumber','pending':'pendingnumber'})


DB_USData_df = new_USData_df[['recordeddate', 'positivenumber', 'negativenumber','pendingnumber']].copy()
DB_USData_df.head()

,recordeddate,positivenumber,negativenumber,pendingnumber
0,2020-12-06,14534035,161986294,13592
1,2020-12-05,14357264,160813704,13433
2,2020-12-04,14146191,159286709,12714
3,2020-12-03,13921360,158026052,15106
4,2020-12-02,13711156,156787587,14368


In [75]:
rds_connection_string = "postgres:<password>@localhost:5432/USvsItalyCOVID20"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [76]:
engine.table_names()

['uscoviddata', 'italycoviddata']

In [77]:
DB_USData_df.to_sql(name='uscoviddata', con=engine, if_exists='append', index=False)

In [60]:
DB_ItalyData_df.to_sql(name='italycoviddata', con=engine, if_exists='append', index=False)

In [78]:
pd.read_sql_query('SELECT SUM(I.positivenumber) as TotalItalyCount FROM italycoviddata I', con=engine).head()

,totalitalycount
0,103058478


In [79]:
pd.read_sql_query('SELECT SUM(I.positivenumber) as TotalUSCount FROM uscoviddata I', con=engine).head()

,totaluscount
0,1268277142


In [83]:
pd.read_sql_query('with italyData as (SELECT RecordedDate, cast( Sum(positivenumber) as int) as ItalyPositiveNumber from italycoviddata group by RecordedDate order by RecordedDate desc) SELECT I.RecordedDate , SUM(I.ItalyPositiveNumber) as ItalyPositiveNumber, SUM(U.positivenumber) as USPositiveNumber From italyData I join uscoviddata U on I.RecordedDate = U.RecordedDate GROUP BY I.RecordedDate, U.RecordedDate Order by I.recordeddate desc', con=engine).head()

,recordeddate,italypositivenumber,uspositivenumber
0,2020-12-06,1728878,14534035
1,2020-12-05,1709991,14357264
2,2020-12-04,1688939,14146191
3,2020-12-03,1664829,13921360
4,2020-12-02,1641610,13711156
